In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Loan_Applications_Dataset.csv to Loan_Applications_Dataset (1).csv


In [ ]:
data = pd.read_csv('Loan_Applications_Dataset.csv')

In [ ]:
# ทำความสะอาดข้อมูล
data['loan_purpose'] = data['loan_purpose'].replace('h0use', 'House')  # แก้ไขข้อผิดพลาดในข้อมูล
data['approved'] = data['approved'].map({'Yes': 1, 'No': 0})  # แปลงค่าเป็น binary

In [ ]:
# แก้ไขค่าที่หายไป
data['income'].fillna(data['income'].median(), inplace=True)
data['loan_amount'].fillna(data['loan_amount'].median(), inplace=True)
data['credit_score'].fillna(data['credit_score'].median(), inplace=True)

<ipython-input-29-4638b2398107>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['income'].fillna(data['income'].median(), inplace=True)
<ipython-input-29-4638b2398107>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [ ]:
# แปลง categorical features เป็น numerical
label_encoder = LabelEncoder()
data['loan_purpose'] = label_encoder.fit_transform(data['loan_purpose'])

In [ ]:
# เลือก features และ target
X = data[['age', 'income', 'loan_amount', 'loan_term', 'loan_purpose', 'credit_score']]
y = data['approved']

In [ ]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# ปรับขนาดข้อมูล
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# สร้างโมเดล
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 361ms/step - accuracy: 0.4688 - loss: 0.7143 - val_accuracy: 0.4375 - val_loss: 0.7476
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.4479 - loss: 0.7218 - val_accuracy: 0.4375 - val_loss: 0.7411
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.4792 - loss: 0.7053 - val_accuracy: 0.4375 - val_loss: 0.7354
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5938 - loss: 0.6800 - val_accuracy: 0.4375 - val_loss: 0.7315
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.5729 - loss: 0.6890 - val_accuracy: 0.5000 - val_loss: 0.7258
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.6042 - loss: 0.6701 - val_accuracy: 0.5000 - val_loss: 0.7214
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.6250 - loss: 0.6621 - val_accuracy: 0.4375 - val_loss: 0.7175
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6771 - loss: 0.6481 - val_accuracy: 0.4375 - val_loss: 0.

In [ ]:
# ประเมินโมเดล
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4500 - loss: 0.7618
Test Accuracy: 0.45


In [ ]:
model.save('loan_approval_model.h5')

In [ ]:
model = load_model('loan_approval_model.h5')

In [ ]:
sample_input = np.array([[65, 20000, 100000, 60, 3, 500]])  # ตัวอย่างข้อมูล input
sample_input_scaled = scaler.transform(sample_input)  # ปรับขนาดข้อมูล
prediction = model.predict(sample_input_scaled)
print(f"ค่าความน่าจะเป็น: {prediction[0][0]:.4f}")  # แสดงค่าความน่าจะเป็นเป็นทศนิยม 4 ตำแหน่ง
result = "อนุมัติ" if prediction[0] > 0.5 else "ไม่อนุมัติ"
print(f"ผลการทำนาย: {result}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
ค่าความน่าจะเป็น: 0.8264
ผลการทำนาย: อนุมัติ


In [ ]:
from google.colab import files
files.download('loan_approval_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>